# Just to hide things

In [141]:
!pip install missingno

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


In [3]:
import pandas as pd
import numpy as np
import missingno as msn

# Further cleaning

In [ ]:
result['start_date'] = pd.to_datetime(result['start_date'], errors='coerce')

In [ ]:
result['end_date'] = pd.to_datetime(result['end_date'], errors='coerce')

In [ ]:
allowed_values = ['Amber', 'Green', 'Red', 'Amber/red', 'Amber/Green']
result['colour_rating_new'] = result['colour_rating_new'].apply(lambda x: x if x in allowed_values else np.nan)

In [ ]:
result = result.rename(columns={'colour_rating_new': 'colour_rating'})

In [ ]:
result['yearly_forecast_new'] = result['yearly_forecast'].combine_first(result['forecast_incl_NGC'])

In [ ]:
result = result.drop(columns=['yearly_forecast', 'forecast_incl_NGC'])

In [ ]:
result = result.rename(columns={'yearly_forecast_new': 'yearly_forecast'})

In [ ]:
result = result.drop(columns=['yearly_budget'])

In [ ]:
result = result.rename(columns={'total_baseline': 'yearly_budget'})
result = result.rename(columns={'TOTAL Baseline Benefits (Łm)': 'total_benefits'})

In [ ]:
column_order = ['project_name','department','colour_rating','description_aims','rating_comment','start_date','end_date','schedule_comment','yearly_budget','yearly_forecast','wlc_baseline_incl_NCG','variance_comment','budget_comment','year','report_category','project_number','total_benefits','benefits_comment']

In [ ]:
result = result[column_order]

In [ ]:
df_macro = pd.read_csv('../data/macro/GDP.csv',encoding='iso-8859-2', delimiter=';')

In [ ]:
df_macro

In [ ]:
df_macro.info()

In [ ]:
df_franzi = pd.read_csv('../data/EDA/2021_2023.csv')

In [ ]:
# Schritt 1: Identifizieren der relevanten Zeilen
remained_scheduled_df = df_franzi[df_franzi['schedule_comment'].str.contains('remained scheduled', case=False)]

# Schritt 2: Aktuellsten 'end_date'-Wert für jedes 'project_name' ermitteln
latest_end_dates = df_franzi.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'end_date'])

# Schritt 3: Zuweisen des aktuellsten 'end_date' an die relevanten Zeilen
for index, row in remained_scheduled_df.iterrows():
    latest_end_date = latest_end_dates[row['project_name']]
    if pd.isna(row['end_date']) and pd.notna(latest_end_date):
        df_franzi.at[index, 'end_date'] = latest_end_date

# Ergebnis anzeigen
df_franzi


In [ ]:
# Schritt 2: Aktuellsten 'end_date'-Wert für jedes 'project_name' ermitteln
latest_end_dates = df_franzi.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'start_date'])

# Schritt 3: Zuweisen des aktuellsten 'end_date' an die relevanten Zeilen
for index, row in remained_scheduled_df.iterrows():
    latest_end_date = latest_end_dates[row['project_name']]
    if pd.isna(row['start_date']) and pd.notna(latest_end_date):
        df_franzi.at[index, 'start_date'] = latest_end_date

# Ergebnis anzeigen
df_franzi

In [ ]:
    month_mapping = {
        'jan': '01', 'january': '01',
        'feb': '02', 'february': '02',
        'mar': '03', 'march': '03',
        'apr': '04', 'april': '04',
        'may': '05', 'may': '05',
        'jun': '06', 'june': '06',
        'jul': '07', 'july': '07',
        'aug': '08', 'august': '08',
        'sep': '09', 'september': '09',
        'oct': '10', 'october': '10',
        'nov': '11', 'november': '11',
        'dec': '12', 'december': '12'
    }



In [ ]:
def extract_date_after_to_or_on(comment):
    REGEX = "(to|is|on)\s*(\d{1,2})\s+(\w+)\s+(\d{2,4})"
    x = re.compile(REGEX)
    
    match_REGEX = x.search(comment)      
    
    if match_REGEX:
        day = match_REGEX.group(2)
        month = match_REGEX.group(3)
        year = match_REGEX.group(4)
        if len(day) == 1:
            day = '0'+ day
        month = month_mapping.get(month.lower())
        if len(year) == 2:
            year = '20' + year
        return year +'-'+ month +'-'+ day
    
for index, row in df_franzi.iterrows():
    if pd.isna(row['end_date']):
        extracted_date = extract_date_after_to_or_on(row['schedule_comment'])
        if extracted_date:
            df_franzi.at[index, 'end_date'] = extracted_date

df_franzi



In [ ]:
# Der Code zum Filtern von Zeilen, in denen 'start_date' oder 'end_date' 'NaN' ist
# und die 'schedule_comment' nicht mit 'Exempt' oder '\r\nExempt' beginnen

nan_dates_df = df_franzi[
    ((df_franzi['end_date'].isna())) & 
    (~df_franzi['schedule_comment'].str.match(r'^(Exempt|\r\nExempt)'))
][['start_date', 'end_date', 'schedule_comment']]

# Ausgabe des gefilterten DataFrames
nan_dates_df

In [ ]:
# Der Code zum Filtern von Zeilen, in denen 'start_date' oder 'end_date' 'NaN' ist
# und die 'schedule_comment' nicht mit 'Exempt' oder '\r\nExempt' beginnen

nan_dates_df_start = df_franzi[
    ((df_franzi['start_date'].isna())) & 
    (~df_franzi['schedule_comment'].str.match(r'^(Exempt|\r\nExempt)'))
][['start_date', 'schedule_comment']]

# Ausgabe des gefilterten DataFrames
nan_dates_df_start

In [171]:
df1 = pd.read_pickle("../data/pickle/cleaned_2021_2023.pkl")
df2 = pd.read_pickle('../data/pickle/cleaned_2018_2020.pkl')
df3 = pd.read_pickle('../data/pickle/cleaned_2014-2017.pkl')
df4 = pd.read_csv('../data/macro/GDP.csv',encoding='iso-8859-2', delimiter=';')

In [172]:
df = pd.concat([df1, df2, df3], axis=0, ignore_index=True)

In [173]:
df = df.merge(df4, on='year', how='left')

In [174]:
df.drop(columns='Unnamed: 0')

,project_name,department,colour_rating,description_aims,rating_comment,start_date,end_date,schedule_comment,yearly_budget,yearly_forecast,...,GDP,annual_earning_ft,unemployment_rate,youth_unemployment_rate,inflation_rate,population,gov_debt,tax_revenue,revenue_excl_grants,grants_and_other_revenue
0,Civil Service Pensions 2015 Remedy,CO,Amber,The 2015 Pensions Remedy Programme was created...,"Compared to financial year 21/22-Q4, the Deliv...",2020-05-20,2024-09-30,"Compared to financial year 21/22-Q4, the proje...",7.90,4.20,...,2273126.0,34963.0,0.0440,0.1254,0.042,68350000.0,22967000.0,7885900.0,NaN,NaN
1,Commercial Capability Expansion Programme,CO,Amber,The original Commercial Capability Programme s...,"Compared to financial year 21/22-Q4, the Seni...",2017-04-01,2023-03-31,"Compared to financial year 21/22-Q4, the proje...",3.70,3.10,...,2273126.0,34963.0,0.0440,0.1254,0.042,68350000.0,22967000.0,7885900.0,NaN,NaN
2,Falcon IT Platform Refresh and Migration,CO,Amber,Falcon is a business change programme that wil...,The Senior Responsible Owner's Delivery Confid...,2022-05-01,2025-03-31,The project's end-date at 22/23-Q4 is 2025-03-...,0.00,0.00,...,2273126.0,34963.0,0.0440,0.1254,0.042,68350000.0,22967000.0,7885900.0,NaN,NaN
3,Future Service Programme,CO,Amber,Future Services Programme is a series of procu...,"Compared to financial year 21/22-Q4, the Deliv...",2020-10-01,2025-12-31,"Compared to financial year 21/22-Q4, the proje...",1.60,1.70,...,2273126.0,34963.0,0.0440,0.1254,0.042,68350000.0,22967000.0,7885900.0,NaN,NaN
4,GOV.UK One Login,CO,Amber,GOV.UK One Login will provide a single account...,"Compared to financial year 21/22-Q4, the Infra...",2021-01-04,2025-03-31,"Compared to financial year 21/22-Q4, the proje...",114.00,114.00,...,2273126.0,34963.0,0.0440,0.1254,0.042,68350000.0,22967000.0,7885900.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1725,IT Transformation Programme,NCA,Amber,IT Transformation is one of three programmes t...,The IPA Review Team made 7 recommendations whe...,2016-04-01 00:00:00,2022-07-31 00:00:00,Schedule on track,14.00,14.00,...,2167415.0,28759.0,0.0433,0.1199,0.027,66058859.0,33584200.0,5327300.0,7075890.0,442790.0
1726,NCA Transformation Programme,NCA,Amber,The NCAÂs Transformation Programme which will...,The current status of the NCA Transformation P...,2014-04-01 00:00:00,2019-03-31 00:00:00,Schedule on track,25.50,25.50,...,2167415.0,28759.0,0.0433,0.1199,0.027,66058859.0,33584200.0,5327300.0,7075890.0,442790.0
1727,Census Transformation Programme,ONS,Amber,The Census Transformation Programme will run f...,1. ONS requires greater clarity from HMT on th...,2015-01-01 00:00:00,2025-03-31 00:00:00,Tranche 1 focussing on 2017 Test on schedule. ...,36.03,36.03,...,2167415.0,28759.0,0.0433,0.1199,0.027,66058859.0,33584200.0,5327300.0,7075890.0,442790.0
1728,Unnamed: 144,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [178]:
df["colour_rating"] = df["colour_rating"].str.lower()

# Drop rows where 'project_name' contains 'Unnamed:' followed by other characters
df = df[~df['project_name'].str.contains(r'Unnamed:.*', na=False)]

# Optionally, reset the index of the cleaned DataFrame
df = df.reset_index(drop=True)

In [179]:
latest_start_dates = df.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'start_date'])

In [181]:
for index, row in df.iterrows():
    latest_start_date = latest_start_dates[row['project_name']]
    if pd.isna(row['start_date']) and pd.notna(latest_start_date):
        df.at[index, 'start_date'] = latest_start_date

KeyError: NaT

In [182]:
latest_benefits = df.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'total_benefits'])

for index, row in df.iterrows():
    latest_benefit = latest_benefits[row['project_name']]
    if pd.isna(row['total_benefits']) and pd.notna(latest_benefit):
        df.at[index, 'total_benefits'] = latest_benefit

KeyError: NaT

In [183]:
latest_categories = df.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'report_category'])

for index, row in df.iterrows():
    latest_category = latest_categories[row['project_name']]
    if pd.isna(row['report_category']) and pd.notna(latest_category):
        df.at[index, 'report_category'] = latest_category

KeyError: NaT

In [184]:
latest_numbers = df.groupby('project_name').apply(lambda x: x.loc[x['year'].idxmax(), 'project_number'])

for index, row in df.iterrows():
    latest_number = latest_numbers[row['project_name']]
    if pd.isna(row['project_number']) and pd.notna(latest_number):
        df.at[index, 'project_number'] = latest_number

KeyError: NaT

In [186]:
df.to_pickle('ready_for_EDA.pkl')

In [187]:
df['description_aims'].unique().tolist()

['The 2015 Pensions Remedy Programme was created to end age discrimination within the Civil Service Pension Schemes and is also creating solutions to remediate any affected to historic members of the scheme.',
 'The original Commercial Capability Programme successfully established the Government Commercial Organisation (GCO) - a single central employer of several hundred Commercial Specialists (Grade 6 and above) for central government departments. The Civil Service Board endorsed proposals that commercial capability building interventions should be extended to Grade 7 commercial professionals within central government, Wider Government Bodies (WGBs) and training and accreditation developed and delivered to the Civil Service contract management community. The Commercial Capability Expansion Programme has been established to impact these new target populations deeper within the Civil Service and more broadly across the Public Sector.',
 'Falcon is a business change programme that will f

In [4]:
df = pd.read_pickle('../data/pickle/cleaned_categorized_department.pkl')

In [5]:
df

,project_name,department,colour_rating,description_aims,rating_comment,start_date,end_date,schedule_comment,yearly_budget,yearly_forecast,...,GDP,annual_earning_ft,unemployment_rate,youth_unemployment_rate,inflation_rate,population,gov_debt,tax_revenue,revenue_excl_grants,grants_and_other_revenue
0,Civil Service Pensions 2015 Remedy,CO,amber,The 2015 Pensions Remedy Programme was created...,"Compared to financial year 21/22-Q4, the Deliv...",2020-05-20,2024-09-30,"Compared to financial year 21/22-Q4, the proje...",7.90,4.20,...,2273126.0,34963.0,0.0440,0.1254,0.042,68350000.0,22967000.0,7885900.0,NaN,NaN
1,Commercial Capability Expansion Programme,CO,amber,The original Commercial Capability Programme s...,"Compared to financial year 21/22-Q4, the Seni...",2017-04-01,2023-03-31,"Compared to financial year 21/22-Q4, the proje...",3.70,3.10,...,2273126.0,34963.0,0.0440,0.1254,0.042,68350000.0,22967000.0,7885900.0,NaN,NaN
2,Falcon IT Platform Refresh and Migration,CO,amber,Falcon is a business change programme that wil...,The Senior Responsible Owner's Delivery Confid...,2022-05-01,2025-03-31,The project's end-date at 22/23-Q4 is 2025-03-...,0.00,0.00,...,2273126.0,34963.0,0.0440,0.1254,0.042,68350000.0,22967000.0,7885900.0,NaN,NaN
3,Future Service Programme,CO,amber,Future Services Programme is a series of procu...,"Compared to financial year 21/22-Q4, the Deliv...",2020-10-01,2025-12-31,"Compared to financial year 21/22-Q4, the proje...",1.60,1.70,...,2273126.0,34963.0,0.0440,0.1254,0.042,68350000.0,22967000.0,7885900.0,NaN,NaN
4,GOV.UK One Login,CO,amber,GOV.UK One Login will provide a single account...,"Compared to financial year 21/22-Q4, the Infra...",2021-01-04,2025-03-31,"Compared to financial year 21/22-Q4, the proje...",114.00,114.00,...,2273126.0,34963.0,0.0440,0.1254,0.042,68350000.0,22967000.0,7885900.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723,Secure Training centre (STC) Retendering Project,MOJ,amber/red,The project is retendering and mobilising a Se...,Quarter 2 was assessed as Amber/Red as there w...,2012-11-05 00:00:00,2017-05-01 00:00:00,The new service at Rainsbrook STC commenced on...,10.83,10.83,...,2167415.0,28759.0,0.0433,0.1199,0.027,66058859.0,33584200.0,5327300.0,7075890.0,442790.0
1724,IT Transformation Programme,NCA,amber,IT Transformation is one of three programmes t...,The IPA Review Team made 7 recommendations whe...,2016-04-01 00:00:00,2022-07-31 00:00:00,Schedule on track,14.00,14.00,...,2167415.0,28759.0,0.0433,0.1199,0.027,66058859.0,33584200.0,5327300.0,7075890.0,442790.0
1725,NCA Transformation Programme,NCA,amber,The NCAÂs Transformation Programme which will...,The current status of the NCA Transformation P...,2014-04-01 00:00:00,2019-03-31 00:00:00,Schedule on track,25.50,25.50,...,2167415.0,28759.0,0.0433,0.1199,0.027,66058859.0,33584200.0,5327300.0,7075890.0,442790.0
1726,Census Transformation Programme,ONS,amber,The Census Transformation Programme will run f...,1. ONS requires greater clarity from HMT on th...,2015-01-01 00:00:00,2025-03-31 00:00:00,Tranche 1 focussing on 2017 Test on schedule. ...,36.03,36.03,...,2167415.0,28759.0,0.0433,0.1199,0.027,66058859.0,33584200.0,5327300.0,7075890.0,442790.0


In [6]:
df_with_dummies = pd.get_dummies(df, columns=['department', 'colour_rating'], drop_first=True)


In [8]:
df_with_dummies.to_pickle('../data/pickle/dummies.pkl')

In [9]:
df = pd.read_pickle('../data/pickle/dummies.pkl')

In [13]:
df["start_date"] = pd.to_datetime(df["start_date"])
df["end_date"] = pd.to_datetime(df["end_date"])
# Calculate the duration in years
df["year_duration"] = (df["end_date"] - df["start_date"]).dt.days / 365.25

# Optionally, round the duration to a specific number of decimal places
df["year_duration"] = df["year_duration"].round(2)

In [14]:
df.columns

Index(['project_name', 'description_aims', 'rating_comment', 'start_date',
       'end_date', 'schedule_comment', 'yearly_budget', 'yearly_forecast',
       'wlc_baseline_incl_NCG', 'variance_comment', 'budget_comment', 'year',
       'report_category', 'project_number', 'total_benefits',
       'benefits_comment', 'GDP', 'annual_earning_ft', 'unemployment_rate',
       'youth_unemployment_rate', 'inflation_rate', 'population', 'gov_debt',
       'tax_revenue', 'revenue_excl_grants', 'grants_and_other_revenue',
       'department_CO', 'department_CPS', 'department_DBT', 'department_DCMS',
       'department_DEFRA', 'department_DEFRA & DFT', 'department_DESNZ',
       'department_DFE', 'department_DFID', 'department_DFT',
       'department_DHSC', 'department_DLUHC', 'department_DSIT',
       'department_DWP', 'department_FCDO', 'department_HMLR',
       'department_HMRC', 'department_HMT', 'department_HO', 'department_MOD',
       'department_MOJ', 'department_NCA', 'department_NS&I', 

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 57 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   project_name               1727 non-null   object        
 1   description_aims           1724 non-null   object        
 2   rating_comment             1723 non-null   object        
 3   start_date                 1581 non-null   datetime64[ns]
 4   end_date                   1617 non-null   datetime64[ns]
 5   schedule_comment           1719 non-null   object        
 6   yearly_budget              1590 non-null   float64       
 7   yearly_forecast            1592 non-null   float64       
 8   wlc_baseline_incl_NCG      1522 non-null   float64       
 9   variance_comment           1682 non-null   object        
 10  budget_comment             1717 non-null   object        
 11  year                       1727 non-null   float64       
 12  report

In [18]:
df.to_pickle('pickle_w_dummies_and_datetime.pkl')

In [16]:
num_columns = df.select_dtypes(exclude=['object']).columns.tolist()

In [17]:
text_columns = df.select_dtypes(include=['object']).columns.tolist()

In [ ]:
df_numeric = df.drop(columns=[(text_columns)], axis=)

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

corr_matrix = df.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)

plt.title('Correlationmatrix')
plt.show()


ValueError: could not convert string to float: 'Civil Service Pensions 2015 Remedy'